In [5]:
import os

In [6]:
original_triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/original_triples/"
mappings_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/mappings/"
filtered_triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples/"

In [ ]:
[elem for elem in os.listdir(original_triples_path) if ".triples" in elem]

In [ ]:
[elem for elem in os.listdir(mappings_path) if ".json" in elem]

In [7]:
edges_to_keep = ["http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLink", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://purl.org/dc/terms/subject", "http://www.w3.org/2004/02/skos/core#broader"]
edges_to_delete = ["http://dbkwik.webdatacommons.org/ontology/Image", "http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLinkText", "/File:"]

In [8]:
files = [elem for elem in os.listdir(original_triples_path) if ".triples" in elem]

for file in files:
    kept = 0
    deleted = 0
    with open(original_triples_path + file, "r") as f, open(filtered_triples_path + file, "w") as g:
        for line in f:
            if any(edge in line for edge in edges_to_keep) and not any(edge in line for edge in edges_to_delete):
                g.write(line)
                kept += 1
            else:
                deleted += 1
    print(f"{file}: {deleted}/{kept + deleted} triples are deleted")

stexpanded.triples: 243826/489249 triples are deleted
marvel.triples: 3488849/5778444 triples are deleted
memoryalpha.triples: 985512/2268717 triples are deleted
swg.triples: 149618/230143 triples are deleted
memorybeta.triples: 848032/2171306 triples are deleted
swtor.triples: 71766/116181 triples are deleted
starwars.triples: 2740396/5774014 triples are deleted
mcu.triples: 770599/1031437 triples are deleted
